# Output Parsers

language models output text.But many times you need more structured information. This is where output parsers come in.
Output oarsers are classes that help structure language model responses. There are two main method an outpur parser must implement:

Get Format Instruction : A method which returns a string  containing instructions for how the output of a language model should be formatted.

Parse : A method which takes in a string ( assumed to be response from language model) and parse it into some structure.

In [2]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
model=GoogleGenerativeAI(model="gemini-1.5-pro-latest",temperature=0)

### CSV Parser

In [5]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt= PromptTemplate(
    template="List five places {places}.\n{format_instructions}",
    input_variables=["places"],
    partial_variables={"format_instructions": format_instructions}
)

In [8]:
chain = prompt | model 
chain.invoke({"places":"for summer tourism in india"})

'Manali, Ladakh, Shimla, Darjeeling, Andaman and Nicobar Islands \n'

In [9]:
chain = prompt | model | output_parser
chain.invoke({"places":"for summer tourism in india"})

['Manali', 'Ladakh', 'Shimla', 'Darjeeling', 'Andaman and Nicobar Islands']

### JSON Parser

In [15]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [16]:
class Travel(BaseModel):
    place: str = Field(description="name of the places")
    description: str = Field(description="description of the place")
    activities:  str = Field(description="what to do in that place")

In [17]:
travel_query="Suggest a place in India for going on a trip this summer to avoid heat."
#setup a parser + inject instruction into prompt template
parser = JsonOutputParser(pydantic_object=Travel)
prompt=PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["places"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | parser

chain.invoke({"query": travel_query})

{'place': 'Manali, Himachal Pradesh',
 'description': 'Nestled in the Himalayas, Manali offers a cool escape from the scorching summer heat. With snow-capped peaks, lush green valleys, and the Beas River flowing through, it provides a refreshing and picturesque setting.',
 'activities': 'Enjoy adventure activities like trekking, paragliding, and river rafting. Explore ancient temples like Hidimba Devi Temple and Manu Temple. Take a scenic drive to Rohtang Pass and Solang Valley. Indulge in local cuisine and shop for handicrafts.'}

#### Without Pydantic

In [18]:
travel_query="Suggest a place in India for going on a trip this summer to avoid heat."
#setup a parser + inject instruction into prompt template
parser = JsonOutputParser()
prompt=PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["places"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [19]:
chain = prompt | model | parser

chain.invoke({"query": travel_query})

{'query': 'Suggest a place in India for going on a trip this summer to avoid heat.',
 'suggestion': 'Manali, Himachal Pradesh',
 'reason': 'Nestled in the Himalayas, Manali offers a cool escape from the scorching summer heat. Enjoy pleasant weather, stunning mountain views, and adventure activities like trekking and paragliding.',
 'alternative_suggestions': ['Shimla, Himachal Pradesh',
  'Darjeeling, West Bengal',
  'Gulmarg, Jammu & Kashmir',
  'Leh Ladakh, Jammu & Kashmir']}

### Structured Output Parser

In [20]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [24]:
response_schemas = [
    ResponseSchema(name="answer", description="answer to the user's question"),
    ResponseSchema(name="description", description="Detailed description to the answer topic"),
    ResponseSchema(name="applications", description="real world application of the answer topic")
]
output_parser=StructuredOutputParser.from_response_schemas(response_schemas)

In [25]:
format_instructions=output_parser.get_format_instructions()
prompt=PromptTemplate(
    template="Answer the user question as best as possible.\n{format_instructions}\n{question}\n",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [26]:
chain = prompt | model | output_parser
chain.invoke({"question":"Name an invention in healthcare that has caused revolution in 21st century"})

{'answer': 'One of the most revolutionary healthcare inventions of the 21st century is **Artificial Intelligence (AI)**.',
 'description': "AI in healthcare encompasses the use of complex algorithms and software to mimic human cognition in analyzing, understanding, and interpreting complex medical data. It's not a single invention but a powerful technology with applications ranging from disease diagnosis and personalized treatment plans to drug discovery and robotic surgery.",
 'applications': 'Here are some real-world applications of AI in healthcare:\n\n* **Medical Imaging Analysis:** AI algorithms can analyze medical images like X-rays, CT scans, and MRIs to detect abnormalities with high accuracy, often surpassing human capabilities.\n* **Drug Discovery and Development:** AI significantly speeds up the process of identifying potential drug candidates and predicting their effectiveness, leading to faster development of new treatments.\n* **Personalized Medicine:** AI analyzes patien